# PACT-Based Multi-Agent Paper Critique System

This notebook implements a multi-agent system for critiquing student papers using the PACT (PennCLO Academic Critique Taxonomy) framework.

## Architecture Overview

The system consists of:
1. **5 Specialized PACT Agents** - Each focused on one dimension of the PACT taxonomy
2. **Supervisor Agent** - Coordinates the specialized agents and synthesizes their feedback
3. **Input Processing** - Handles student paper submission and parsing
4. **Output Generation** - Creates comprehensive, structured feedback reports

### PACT Dimensions:
1. **Research Foundations** - Problem definition, frameworks, literature
2. **Methodological Rigor** - Methods, data, analysis, limitations
3. **Structure & Coherence** - Organization, flow, transitions
4. **Academic Precision** - Terms, citations, grammar, formatting
5. **Critical Sophistication** - Reflexivity, originality, theoretical depth

In [ ]:
# Load environment variables and set up auto-reload
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

## Load and Parse PACT Taxonomy

In [ ]:
%%writefile ../src/pact/pact_taxonomy.py

"""
PACT Taxonomy Loader and Parser

This module loads and structures the PACT taxonomy for use by the critique agents.
"""

import json
import zipfile
import xml.etree.ElementTree as ET
from typing import Dict, Any, List
from pathlib import Path

def load_pact_taxonomy(file_path: str = "../PACT_JSON.docx") -> Dict[str, Any]:
    """
    Load the PACT taxonomy from the docx file.
    
    Returns a structured dictionary with the taxonomy dimensions.
    """
    # Check if we already have a parsed JSON version
    json_path = Path("../pact_taxonomy.json")
    if json_path.exists():
        with open(json_path, 'r') as f:
            return json.load(f)
    
    # Otherwise, parse from docx
    with zipfile.ZipFile(file_path, 'r') as docx:
        xml_content = docx.read('word/document.xml')
        tree = ET.fromstring(xml_content)
        
        namespace = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
        paragraphs = []
        for para in tree.iter(namespace + 'p'):
            texts = [node.text for node in para.iter(namespace + 't') if node.text]
            if texts:
                paragraphs.append(''.join(texts))
        
        # Parse JSON from text
        full_text = '\n'.join(paragraphs)
        json_start = full_text.find('{')
        if json_start != -1:
            json_text = full_text[json_start:]
            # Find matching closing brace
            brace_count = 0
            end_pos = 0
            for i, char in enumerate(json_text):
                if char == '{':
                    brace_count += 1
                elif char == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        end_pos = i + 1
                        break
            
            json_text = json_text[:end_pos]
            pact_data = json.loads(json_text)
            
            # Save for future use
            with open(json_path, 'w') as f:
                json.dump(pact_data, f, indent=2)
            
            return pact_data
    
    raise ValueError("Could not parse PACT taxonomy from file")

def get_dimension_details(pact_data: Dict[str, Any], dimension_id: str) -> Dict[str, Any]:
    """
    Extract details for a specific PACT dimension.
    
    Args:
        pact_data: The full PACT taxonomy data
        dimension_id: The dimension ID (e.g., "1.0.0")
    
    Returns:
        Dictionary with dimension details including subsections
    """
    dimensions = pact_data.get('dimensions', {})
    return dimensions.get(dimension_id, {})

def get_all_dimensions(pact_data: Dict[str, Any]) -> List[tuple]:
    """
    Get all main dimensions from the PACT taxonomy.
    
    Returns:
        List of (dimension_id, dimension_name, dimension_data) tuples
    """
    dimensions = pact_data.get('dimensions', {})
    main_dimensions = []
    
    for dim_id in ['1.0.0', '2.0.0', '3.0.0', '4.0.0', '5.0.0']:
        if dim_id in dimensions:
            dim_data = dimensions[dim_id]
            main_dimensions.append((dim_id, dim_data.get('name'), dim_data))
    
    return main_dimensions

## Define State and Schemas for Critique System

In [ ]:
%%writefile ../src/pact/state_pact_critique.py

"""
State Definitions and Schemas for PACT Critique System

This module defines the state objects and structured schemas used for
the multi-agent paper critique workflow.
"""

import operator
from typing import Optional, List, Dict, Any
from typing_extensions import Annotated

from pydantic import BaseModel, Field
from langgraph.graph import MessagesState

# ===== STATE DEFINITIONS =====

class PaperCritiqueState(MessagesState):
    """
    Main state for the PACT critique system.
    
    Tracks the paper being critiqued, individual agent feedback,
    and the final synthesized critique.
    """
    # The student paper to critique
    paper_content: str
    
    # Paper metadata
    paper_title: Optional[str] = None
    paper_type: Optional[str] = None  # thesis, dissertation, article, etc.
    
    # Individual dimension critiques from specialized agents
    dimension_critiques: Annotated[Dict[str, Any], operator.add] = {}
    
    # Supervisor's analysis plan
    critique_plan: Optional[str] = None
    
    # Final synthesized critique
    final_critique: Optional[str] = None
    
    # Overall paper score (0-100)
    overall_score: Optional[float] = None
    
    # Priority areas for improvement
    priority_improvements: List[str] = []

# ===== STRUCTURED OUTPUT SCHEMAS =====

class DimensionCritique(BaseModel):
    """
    Schema for individual dimension critiques from specialized agents.
    """
    dimension_id: str = Field(
        description="The PACT dimension ID (e.g., '1.0.0')"
    )
    dimension_name: str = Field(
        description="The dimension name (e.g., 'Research Foundations')"
    )
    strengths: List[str] = Field(
        description="Specific strengths identified in this dimension",
        default_factory=list
    )
    weaknesses: List[str] = Field(
        description="Specific weaknesses or areas for improvement",
        default_factory=list
    )
    specific_issues: List[Dict[str, str]] = Field(
        description="Specific issues with location and severity",
        default_factory=list
    )
    recommendations: List[str] = Field(
        description="Actionable recommendations for improvement",
        default_factory=list
    )
    rubric_scores: Dict[str, int] = Field(
        description="Rubric scores for subsections (1-5 scale)",
        default_factory=dict
    )
    dimension_score: float = Field(
        description="Overall score for this dimension (0-100)",
        ge=0, le=100
    )
    severity: str = Field(
        description="Overall severity level: Critical, Major, Moderate, Minor",
        default="Moderate"
    )

class CritiquePlan(BaseModel):
    """
    Schema for the supervisor's critique plan.
    """
    paper_summary: str = Field(
        description="Brief summary of the paper's content and purpose"
    )
    initial_assessment: str = Field(
        description="Initial high-level assessment of paper quality"
    )
    dimensions_to_evaluate: List[str] = Field(
        description="List of PACT dimensions to evaluate",
        default_factory=lambda: ['1.0.0', '2.0.0', '3.0.0', '4.0.0', '5.0.0']
    )
    special_considerations: List[str] = Field(
        description="Any special considerations for this particular paper",
        default_factory=list
    )

class FinalCritique(BaseModel):
    """
    Schema for the final synthesized critique.
    """
    executive_summary: str = Field(
        description="Executive summary of the critique"
    )
    overall_assessment: str = Field(
        description="Overall assessment of the paper's quality"
    )
    dimension_summaries: Dict[str, str] = Field(
        description="Summary for each PACT dimension evaluated",
        default_factory=dict
    )
    key_strengths: List[str] = Field(
        description="Top 3-5 key strengths across all dimensions",
        default_factory=list
    )
    priority_improvements: List[str] = Field(
        description="Top 3-5 priority areas for improvement",
        default_factory=list
    )
    actionable_next_steps: List[str] = Field(
        description="Specific, actionable next steps for the author",
        default_factory=list
    )
    overall_score: float = Field(
        description="Overall paper score (0-100)",
        ge=0, le=100
    )
    recommendation: str = Field(
        description="Final recommendation: Accept, Revise, Major Revision, Reject"
    )

## Create Specialized PACT Dimension Agents

In [ ]:
%%writefile ../src/pact/pact_dimension_agents.py

"""
Enhanced PACT Dimension Critique Agents with Deep Analysis

This module implements individual agents for each PACT dimension,
providing comprehensive, detailed analysis matching professional assessment quality.
"""

from typing import Dict, Any, List
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

from pact.state_pact_critique import PaperCritiqueState
from pact.enhanced_schemas import DetailedDimensionCritique, Issue
from pact.pact_taxonomy import load_pact_taxonomy, get_dimension_details

# Initialize model for critique agents with higher quality
critique_model = init_chat_model(model="openai:gpt-4o", temperature=0.2)

def create_enhanced_dimension_prompt(paper_content: str, dimension_data: Dict[str, Any]) -> str:
    """
    Create a comprehensive critique prompt for detailed PACT dimension analysis.
    """
    return f"""
You are an expert academic reviewer specializing in the '{dimension_data.get('name')}' dimension of academic writing.

Provide a COMPREHENSIVE, PROFESSIONAL-GRADE critique matching the depth of analysis in the PACT Analysis Report standard.

DIMENSION: {dimension_data.get('name')} ({dimension_data.get('id', 'N/A')})
Description: {dimension_data.get('description')}

EVALUATION FRAMEWORK:
{format_detailed_criteria(dimension_data)}

PAPER TO ANALYZE:
---
{paper_content[:10000]}  # Extended context for deeper analysis
---

REQUIRED ANALYSIS STRUCTURE:

1. **COMPREHENSIVE ASSESSMENT** (300-400 words):
   - Overall evaluation of this dimension's execution in the paper
   - Identify patterns, recurring issues, and systemic strengths/weaknesses
   - Place the work in context of academic standards for this type of document
   - Assess readiness level: submission-ready, minor revisions needed, or major work required

2. **DETAILED STRENGTHS** (minimum 3-5, each 50-100 words):
   For each strength provide:
   - Clear identification of what was done well
   - Specific textual evidence (quotes or paraphrases with location)
   - Explanation of why this matters for academic quality
   - How it contributes to the paper's overall effectiveness

3. **CRITICAL ISSUES** (minimum 4-6 structured issues):
   For each issue provide:
   - Title: Clear, specific issue name
   - Location: Precise paragraph/section reference
   - Evidence: Direct quotes showing the problem
   - Why it matters: Academic principle or standard being violated
   - Suggested rewrite: Concrete example of improvement
   - Priority: Critical/High/Medium/Low

4. **SUBSECTION EVALUATION**:
   Analyze each subsection within this dimension:
   - Score (1-5) with detailed justification
   - Specific examples of excellence or deficiency
   - Targeted recommendations for that subsection

5. **ACTIONABLE RECOMMENDATIONS** (minimum 5):
   Prioritized, specific steps including:
   - What exactly to do
   - How to implement it
   - Expected impact on paper quality
   - Examples or templates where applicable

6. **SCORING**:
   - Dimension Score: 0-100 with justification
   - Qualitative Assessment: Inadequate/Developing/Competent/Strong/Exemplary
   - Detailed rationale referencing specific rubric criteria

Focus on academic rigor while remaining constructive. Provide the depth expected in professional peer review.
"""

def format_detailed_criteria(dimension_data: Dict[str, Any]) -> str:
    """
    Format comprehensive evaluation criteria with detection patterns.
    """
    criteria = []
    sections = dimension_data.get('sections', {})
    
    for section_id, section_data in sections.items():
        criteria.append(f"\n{section_id}: {section_data.get('name')}")
        criteria.append(f"   Purpose: {section_data.get('description', 'N/A')}")
        
        subsections = section_data.get('subsections', {})
        for sub_id, sub_data in subsections.items():
            criteria.append(f"\n   {sub_id}: {sub_data.get('name')}")
            
            # Include evaluation criteria
            if 'evaluation_criteria' in sub_data:
                criteria.append(f"      Criteria: {sub_data['evaluation_criteria']}")
            
            # Include detection patterns
            patterns = sub_data.get('detection_patterns', [])
            if patterns:
                criteria.append(f"      Look for: {'; '.join(patterns)}")
            
            # Include quality indicators
            if 'quality_indicators' in sub_data:
                criteria.append(f"      Excellence indicators: {sub_data['quality_indicators']}")
    
    return '\n'.join(criteria)

async def critique_dimension_enhanced(state: PaperCritiqueState, dimension_id: str) -> Dict[str, Any]:
    """
    Perform enhanced critique for a PACT dimension with detailed analysis.
    """
    # Load PACT taxonomy
    pact_data = load_pact_taxonomy()
    dimension_data = get_dimension_details(pact_data, dimension_id)
    
    if not dimension_data:
        raise ValueError(f"Dimension {dimension_id} not found in PACT taxonomy")
    
    # Add ID to dimension data
    dimension_data['id'] = dimension_id
    
    # Create enhanced prompt
    prompt = create_enhanced_dimension_prompt(state['paper_content'], dimension_data)
    
    # Get structured critique from model
    structured_model = critique_model.with_structured_output(DetailedDimensionCritique)
    critique = await structured_model.ainvoke([HumanMessage(content=prompt)])
    
    # Ensure dimension info is properly set
    critique.dimension_id = dimension_id
    critique.dimension_name = dimension_data.get('name', '')
    
    return critique.dict()

# Enhanced agent functions for each dimension
async def critique_research_foundations(state: PaperCritiqueState) -> Dict[str, Any]:
    """
    Agent 1: Deep analysis of Research Foundations (1.0.0)
    
    Evaluates: Problem identification, theoretical framework, literature review,
    research questions, hypotheses, and scholarly positioning.
    """
    critique = await critique_dimension_enhanced(state, "1.0.0")
    return {"dimension_critiques": {"1.0.0": critique}}

async def critique_methodological_rigor(state: PaperCritiqueState) -> Dict[str, Any]:
    """
    Agent 2: Deep analysis of Methodological Rigor (2.0.0)
    
    Evaluates: Research design, data collection, analysis methods,
    validity, reliability, ethics, and limitations.
    """
    critique = await critique_dimension_enhanced(state, "2.0.0")
    return {"dimension_critiques": {"2.0.0": critique}}

async def critique_structure_coherence(state: PaperCritiqueState) -> Dict[str, Any]:
    """
    Agent 3: Deep analysis of Structure & Coherence (3.0.0)
    
    Evaluates: Organization, logical flow, transitions, paragraph structure,
    introduction/conclusion quality, and overall coherence.
    """
    critique = await critique_dimension_enhanced(state, "3.0.0")
    return {"dimension_critiques": {"3.0.0": critique}}

async def critique_academic_precision(state: PaperCritiqueState) -> Dict[str, Any]:
    """
    Agent 4: Deep analysis of Academic Precision (4.0.0)
    
    Evaluates: Technical terminology, citation accuracy, grammar,
    formatting, academic style, and professional presentation.
    """
    critique = await critique_dimension_enhanced(state, "4.0.0")
    return {"dimension_critiques": {"4.0.0": critique}}

async def critique_critical_sophistication(state: PaperCritiqueState) -> Dict[str, Any]:
    """
    Agent 5: Deep analysis of Critical Sophistication (5.0.0)
    
    Evaluates: Critical thinking, theoretical depth, originality,
    reflexivity, nuanced argumentation, and scholarly maturity.
    """
    critique = await critique_dimension_enhanced(state, "5.0.0")
    return {"dimension_critiques": {"5.0.0": critique}}

## Create the Supervisor Agent

In [ ]:
%%writefile ../src/pact/pact_supervisor.py

"""
PACT Critique Supervisor Agent

This module implements the supervisor agent that coordinates the specialized
PACT dimension agents and synthesizes their feedback into a cohesive critique.
"""

from typing import Dict, Any, Literal
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langgraph.types import Command

from pact.state_pact_critique import (
    PaperCritiqueState, CritiquePlan, FinalCritique
)

# Initialize supervisor model
supervisor_model = init_chat_model(model="openai:gpt-4.1", temperature=0.1)

def create_planning_prompt(paper_content: str) -> str:
    """
    Create a prompt for the supervisor to plan the critique.
    """
    return f"""
You are the lead reviewer coordinating a comprehensive academic paper critique using the PACT taxonomy.

Review the following paper and create a critique plan:

PAPER:
---
{paper_content[:3000]}  # Show first part for initial assessment
---

Create a plan that includes:
1. A brief summary of the paper's content and purpose
2. An initial assessment of overall quality and key areas of concern
3. Which PACT dimensions are most relevant to evaluate (all 5 by default)
4. Any special considerations for this particular paper

The PACT dimensions are:
1.0.0 - Research Foundations (problem, framework, literature)
2.0.0 - Methodological Rigor (methods, data, analysis)
3.0.0 - Structure & Coherence (organization, flow, transitions)
4.0.0 - Academic Precision (terms, citations, grammar)
5.0.0 - Critical Sophistication (reflexivity, originality, theory)
"""

def create_synthesis_prompt(state: PaperCritiqueState) -> str:
    """
    Create a prompt for synthesizing all dimension critiques.
    """
    # Format dimension critiques
    critiques_text = ""
    for dim_id, critique in state['dimension_critiques'].items():
        critiques_text += f"\n\n--- {critique['dimension_name']} ({dim_id}) ---\n"
        critiques_text += f"Score: {critique['dimension_score']}/100\n"
        critiques_text += f"Severity: {critique['severity']}\n"
        
        if critique['strengths']:
            critiques_text += f"Strengths:\n"
            for strength in critique['strengths']:
                critiques_text += f"  • {strength}\n"
        
        if critique['weaknesses']:
            critiques_text += f"Weaknesses:\n"
            for weakness in critique['weaknesses']:
                critiques_text += f"  • {weakness}\n"
        
        if critique['recommendations']:
            critiques_text += f"Recommendations:\n"
            for rec in critique['recommendations']:
                critiques_text += f"  • {rec}\n"
    
    return f"""
You are synthesizing feedback from multiple expert reviewers into a cohesive, actionable critique.

CRITIQUE PLAN:
{state.get('critique_plan', 'No plan available')}

INDIVIDUAL DIMENSION CRITIQUES:
{critiques_text}

Create a comprehensive final critique that:
1. Provides an executive summary of the paper's overall quality
2. Synthesizes feedback across all dimensions
3. Identifies the top 3-5 key strengths
4. Identifies the top 3-5 priority areas for improvement
5. Provides specific, actionable next steps
6. Calculates an overall score (weighted average of dimension scores)
7. Makes a final recommendation (Accept, Revise, Major Revision, Reject)

Be constructive and supportive while maintaining academic rigor.
Focus on helping the author improve their work.
"""

async def plan_critique(state: PaperCritiqueState) -> Command[Literal["evaluate_dimensions"]]:
    """
    Supervisor plans the critique approach.
    """
    # Create planning prompt
    prompt = create_planning_prompt(state['paper_content'])
    
    # Get structured plan from model
    structured_model = supervisor_model.with_structured_output(CritiquePlan)
    plan = await structured_model.ainvoke([HumanMessage(content=prompt)])
    
    # Format plan as string for state
    plan_text = f"""
Paper Summary: {plan.paper_summary}

Initial Assessment: {plan.initial_assessment}

Dimensions to Evaluate: {', '.join(plan.dimensions_to_evaluate)}

Special Considerations:
{"; ".join(plan.special_considerations) if plan.special_considerations else "None"}
"""
    
    return Command(
        goto="evaluate_dimensions",
        update={"critique_plan": plan_text}
    )

async def synthesize_critique(state: PaperCritiqueState) -> Dict[str, Any]:
    """
    Supervisor synthesizes all dimension critiques into final feedback.
    """
    # Create synthesis prompt
    prompt = create_synthesis_prompt(state)
    
    # Get structured final critique from model
    structured_model = supervisor_model.with_structured_output(FinalCritique)
    final_critique = await structured_model.ainvoke([HumanMessage(content=prompt)])
    
    # Format final critique as markdown
    critique_text = f"""
# Academic Paper Critique Report

## Executive Summary
{final_critique.executive_summary}

## Overall Assessment
{final_critique.overall_assessment}

**Overall Score:** {final_critique.overall_score}/100
**Recommendation:** {final_critique.recommendation}

## Dimension Evaluations
"""
    
    for dim_id, summary in final_critique.dimension_summaries.items():
        critique_text += f"\n### {dim_id}
{summary}\n"
    
    critique_text += f"""
## Key Strengths
"""
    for strength in final_critique.key_strengths:
        critique_text += f"- {strength}\n"
    
    critique_text += f"""
## Priority Areas for Improvement
"""
    for improvement in final_critique.priority_improvements:
        critique_text += f"- {improvement}\n"
    
    critique_text += f"""
## Actionable Next Steps
"""
    for i, step in enumerate(final_critique.actionable_next_steps, 1):
        critique_text += f"{i}. {step}\n"
    
    return {
        "final_critique": critique_text,
        "overall_score": final_critique.overall_score,
        "priority_improvements": final_critique.priority_improvements,
        "messages": [f"Critique complete. Overall score: {final_critique.overall_score}/100"]
    }

## Build the Complete PACT Critique Workflow

In [ ]:
%%writefile ../src/pact/pact_critique_agent.py

"""
PACT-Based Multi-Agent Paper Critique System

This module integrates all components of the PACT critique system:
- Input processing and paper parsing
- Supervisor planning and coordination
- Parallel evaluation by specialized dimension agents
- Synthesis of feedback into comprehensive critique
"""

from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage

from pact.state_pact_critique import PaperCritiqueState
from pact.pact_supervisor import plan_critique, synthesize_critique
from pact.pact_dimension_agents import (
    critique_research_foundations,
    critique_methodological_rigor,
    critique_structure_coherence,
    critique_academic_precision,
    critique_critical_sophistication
)

# ===== INPUT PROCESSING =====

def process_paper_input(state: MessagesState) -> dict:
    """
    Process the input paper from user messages.
    
    Extracts the paper content and any metadata provided.
    """
    # Get the last user message which should contain the paper
    messages = state.get('messages', [])
    if not messages:
        raise ValueError("No paper provided for critique")
    
    # Extract paper content from the last message
    last_message = messages[-1]
    if isinstance(last_message, HumanMessage):
        paper_content = last_message.content
    else:
        paper_content = str(last_message)
    
    # Extract title if provided in a specific format
    paper_title = None
    if paper_content.startswith("Title:"):
        lines = paper_content.split('\n')
        paper_title = lines[0].replace("Title:", "").strip()
    
    return {
        "paper_content": paper_content,
        "paper_title": paper_title
    }

# ===== PARALLEL DIMENSION EVALUATION =====

async def evaluate_dimensions(state: PaperCritiqueState) -> dict:
    """
    Coordinate parallel evaluation of all PACT dimensions.
    
    This node spawns all dimension agents to work in parallel.
    """
    # Run all dimension critiques in parallel
    import asyncio
    
    tasks = [
        critique_research_foundations(state),
        critique_methodological_rigor(state),
        critique_structure_coherence(state),
        critique_academic_precision(state),
        critique_critical_sophistication(state)
    ]
    
    # Wait for all tasks to complete
    results = await asyncio.gather(*tasks)
    
    # Combine all dimension critiques
    combined_critiques = {}
    for result in results:
        if 'dimension_critiques' in result:
            combined_critiques.update(result['dimension_critiques'])
    
    return {"dimension_critiques": combined_critiques}

# ===== GRAPH CONSTRUCTION =====

def build_pact_critique_graph():
    """
    Build the complete PACT critique workflow graph.
    """
    # Create the workflow
    workflow = StateGraph(PaperCritiqueState)
    
    # Add nodes
    workflow.add_node("process_input", process_paper_input)
    workflow.add_node("plan_critique", plan_critique)
    workflow.add_node("evaluate_dimensions", evaluate_dimensions)
    workflow.add_node("synthesize_critique", synthesize_critique)
    
    # Add edges
    workflow.add_edge(START, "process_input")
    workflow.add_edge("process_input", "plan_critique")
    # plan_critique uses Command to go to evaluate_dimensions
    workflow.add_edge("evaluate_dimensions", "synthesize_critique")
    workflow.add_edge("synthesize_critique", END)
    
    return workflow

# Compile the workflow
pact_critique_builder = build_pact_critique_graph()
pact_critique_agent = pact_critique_builder.compile()

## Test the PACT Critique System

In [ ]:
# Compile and visualize the workflow
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver
from pact.pact_critique_agent import pact_critique_builder

checkpointer = InMemorySaver()
pact_agent = pact_critique_builder.compile(checkpointer=checkpointer)
display(Image(pact_agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Test with a sample paper excerpt
sample_paper = """
Title: The Impact of Social Media on Academic Performance: A Mixed Methods Study

Abstract:
This study examines the relationship between social media usage and academic performance among 
undergraduate students. Using surveys and interviews with 200 students, we found that excessive 
social media use correlates with lower GPA scores. However, educational use of social media 
platforms showed positive effects on collaborative learning.

Introduction:
Social media has become ubiquitous in student life. Many educators worry about its impact on 
academic performance. This study investigates whether these concerns are justified. Previous 
research has shown mixed results, with some studies finding negative correlations and others 
finding no significant relationship. Our research aims to clarify these conflicting findings.

Literature Review:
Smith (2020) found that students who spend more than 3 hours daily on social media have lower 
grades. Jones et al. (2019) argued that the type of social media use matters more than duration. 
Educational platforms like LinkedIn showed different patterns than entertainment-focused platforms 
like TikTok. However, these studies used different methodologies, making comparisons difficult.

Methodology:
We surveyed 200 undergraduate students about their social media habits and collected their GPA 
data. Additionally, we conducted 20 in-depth interviews to understand usage patterns. The survey 
included questions about daily usage time, platform preferences, and purposes of use.

Results:
Students using social media for more than 4 hours daily had an average GPA of 2.8, compared to 
3.2 for those using it less than 2 hours. However, students who used educational features had 
higher engagement scores in group projects.

Discussion:
Our findings suggest a nuanced relationship between social media and academic performance. While 
excessive recreational use appears detrimental, educational applications show promise. Universities 
should consider developing guidelines that encourage productive social media use.

Conclusion:
Social media's impact on academic performance depends on how it's used. Future research should 
explore interventions that promote beneficial usage patterns while minimizing distractions.
"""

# Run the critique
from langchain_core.messages import HumanMessage

thread = {"configurable": {"thread_id": "test_paper_1", "recursion_limit": 30}}
result = await pact_agent.ainvoke(
    {"messages": [HumanMessage(content=sample_paper)]}, 
    config=thread
)

In [ ]:
# Display the final critique
from rich.markdown import Markdown
if 'final_critique' in result:
    display(Markdown(result['final_critique']))
    print(f"\nOverall Score: {result.get('overall_score', 'N/A')}/100")
else:
    print("Critique not yet complete. Check state:", result.keys())

## Advanced Features: File Upload and Batch Processing

In [ ]:
%%writefile ../src/pact/pact_file_processor.py

"""
File Processing Utilities for PACT Critique System

Handles various file formats for paper submission.
"""

import os
from pathlib import Path
from typing import Optional
import PyPDF2
import docx

def read_paper_from_file(file_path: str) -> str:
    """
    Read paper content from various file formats.
    
    Supports: .txt, .pdf, .docx, .md
    """
    path = Path(file_path)
    
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")
    
    extension = path.suffix.lower()
    
    if extension == '.txt' or extension == '.md':
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    
    elif extension == '.pdf':
        text = ""
        with open(path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    
    elif extension == '.docx':
        doc = docx.Document(path)
        return '\n'.join([paragraph.text for paragraph in doc.paragraphs])
    
    else:
        raise ValueError(f"Unsupported file format: {extension}")

def save_critique_to_file(critique: str, output_path: str, format: str = 'md') -> str:
    """
    Save critique to file in specified format.
    """
    path = Path(output_path)
    
    if format == 'md':
        path = path.with_suffix('.md')
        with open(path, 'w', encoding='utf-8') as f:
            f.write(critique)
    
    elif format == 'txt':
        path = path.with_suffix('.txt')
        # Convert markdown to plain text (basic conversion)
        plain_text = critique.replace('#', '').replace('*', '').replace('_', '')
        with open(path, 'w', encoding='utf-8') as f:
            f.write(plain_text)
    
    elif format == 'html':
        path = path.with_suffix('.html')
        import markdown
        html_content = markdown.markdown(critique)
        with open(path, 'w', encoding='utf-8') as f:
            f.write(f"""<!DOCTYPE html>
<html>
<head>
    <title>PACT Critique Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #333; }}
        h2 {{ color: #666; }}
        h3 {{ color: #888; }}
    </style>
</head>
<body>
{html_content}
</body>
</html>""")
    
    return str(path)

async def critique_paper_file(file_path: str, output_dir: Optional[str] = None) -> str:
    """
    Critique a paper from a file and save results.
    """
    from pact.pact_critique_agent import pact_critique_agent
    from langchain_core.messages import HumanMessage
    
    # Read paper content
    paper_content = read_paper_from_file(file_path)
    
    # Run critique
    result = await pact_critique_agent.ainvoke(
        {"messages": [HumanMessage(content=paper_content)]}
    )
    
    # Save critique if output directory specified
    if output_dir and 'final_critique' in result:
        output_path = Path(output_dir) / f"{Path(file_path).stem}_critique"
        saved_path = save_critique_to_file(
            result['final_critique'], 
            str(output_path), 
            format='md'
        )
        print(f"Critique saved to: {saved_path}")
    
    return result.get('final_critique', 'Critique generation failed')

In [ ]:
%%writefile ../src/pact/visualization.py

"""
Visualization Components for PACT Critique System

This module provides spider chart and other visualizations for PACT analysis.
"""

import json
import base64
from typing import Dict, List, Any
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO

def create_spider_chart(dimension_scores: Dict[str, float], 
                        dimension_names: Dict[str, str] = None) -> str:
    """
    Create a spider/radar chart for PACT dimension scores.
    
    Args:
        dimension_scores: Dictionary mapping dimension IDs to scores (0-100)
        dimension_names: Optional custom names for dimensions
        
    Returns:
        Base64 encoded image string for embedding in HTML
    """
    # Default dimension names
    if dimension_names is None:
        dimension_names = {
            "1.0.0": "Research\nFoundations",
            "2.0.0": "Methodological\nRigor",
            "3.0.0": "Structure &\nCoherence",
            "4.0.0": "Academic\nPrecision",
            "5.0.0": "Critical\nSophistication"
        }
    
    # Prepare data
    categories = []
    scores = []
    
    for dim_id in ["1.0.0", "2.0.0", "3.0.0", "4.0.0", "5.0.0"]:
        categories.append(dimension_names.get(dim_id, f"Dimension {dim_id}"))
        scores.append(dimension_scores.get(dim_id, 0))
    
    # Number of variables
    N = len(categories)
    
    # Compute angle for each axis
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    scores += scores[:1]  # Complete the circle
    angles += angles[:1]
    
    # Initialize the spider plot
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
    
    # Draw the plot
    ax.plot(angles, scores, 'o-', linewidth=2, color='#667eea', label='Score')
    ax.fill(angles, scores, alpha=0.25, color='#667eea')
    
    # Fix axis to go in the right order and start at 12 o'clock
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    
    # Set labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, size=10)
    
    # Set y-axis limits and labels
    ax.set_ylim(0, 100)
    ax.set_yticks([20, 40, 60, 80, 100])
    ax.set_yticklabels(['20', '40', '60', '80', '100'], size=8)
    
    # Add grid
    ax.grid(True)
    
    # Add title
    plt.title('PACT Dimension Analysis', size=14, weight='bold', pad=20)
    
    # Add legend with average score
    avg_score = np.mean(scores[:-1])
    ax.legend([f'Overall: {avg_score:.1f}/100'], loc='upper right', bbox_to_anchor=(1.2, 1.1))
    
    # Save to base64 string
    buffer = BytesIO()
    plt.savefig(buffer, format='png', dpi=100, bbox_inches='tight', transparent=True)
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.getvalue()).decode()
    plt.close()
    
    return f"data:image/png;base64,{image_base64}"

def create_subsection_heatmap(subsection_scores: Dict[str, Dict[str, float]]) -> str:
    """
    Create a heatmap visualization for detailed subsection scores.
    
    Args:
        subsection_scores: Nested dict of dimension -> subsection -> score
        
    Returns:
        Base64 encoded heatmap image
    """
    import seaborn as sns
    
    # Prepare data matrix
    dimensions = sorted(subsection_scores.keys())
    subsections = set()
    for subs in subsection_scores.values():
        subsections.update(subs.keys())
    subsections = sorted(subsections)
    
    # Create matrix
    matrix = []
    for dim in dimensions:
        row = []
        for sub in subsections:
            row.append(subsection_scores[dim].get(sub, 0))
        matrix.append(row)
    
    # Create heatmap
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.heatmap(matrix, 
                xticklabels=subsections, 
                yticklabels=dimensions,
                annot=True, 
                fmt='.1f',
                cmap='RdYlGn',
                vmin=0, 
                vmax=100,
                cbar_kws={'label': 'Score'})
    
    plt.title('Detailed Subsection Analysis', size=14, weight='bold')
    plt.xlabel('Subsections', size=12)
    plt.ylabel('Dimensions', size=12)
    plt.tight_layout()
    
    # Save to base64
    buffer = BytesIO()
    plt.savefig(buffer, format='png', dpi=100, bbox_inches='tight')
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.getvalue()).decode()
    plt.close()
    
    return f"data:image/png;base64,{image_base64}"

def generate_html_with_charts(critique_data: Dict[str, Any]) -> str:
    """
    Generate an HTML report with embedded visualizations.
    
    Args:
        critique_data: Complete critique data including scores
        
    Returns:
        HTML string with embedded charts
    """
    # Extract dimension scores
    dimension_scores = {}
    dimension_names = {}
    
    for dim_id, critique in critique_data.get('dimension_critiques', {}).items():
        dimension_scores[dim_id] = critique.get('dimension_score', 0)
        dimension_names[dim_id] = critique.get('dimension_name', f'Dimension {dim_id}')
    
    # Generate spider chart
    spider_chart = create_spider_chart(dimension_scores, dimension_names)
    
    # Generate HTML
    html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>PACT Analysis Report</title>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background: #f5f5f5;
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 30px;
        }}
        .spider-chart {{
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            text-align: center;
            margin-bottom: 30px;
        }}
        .dimension-card {{
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }}
        .score-badge {{
            display: inline-block;
            padding: 5px 15px;
            border-radius: 20px;
            font-weight: bold;
            margin-left: 10px;
        }}
        .score-high {{ background: #4caf50; color: white; }}
        .score-medium {{ background: #ff9800; color: white; }}
        .score-low {{ background: #f44336; color: white; }}
        .issues-list {{
            list-style-type: none;
            padding-left: 0;
        }}
        .issue-item {{
            padding: 10px;
            margin: 10px 0;
            border-left: 4px solid #667eea;
            background: #f8f9fa;
        }}
        .priority-critical {{ border-left-color: #f44336; }}
        .priority-high {{ border-left-color: #ff9800; }}
        .priority-medium {{ border-left-color: #ffc107; }}
        .priority-low {{ border-left-color: #4caf50; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>PACT Comprehensive Analysis Report</h1>
        <p>Generated: {critique_data.get('timestamp', 'N/A')}</p>
        <p>Overall Score: <strong>{critique_data.get('overall_score', 0):.1f}/100</strong></p>
    </div>
    
    <div class="spider-chart">
        <h2>PACT Dimensions Overview</h2>
        <img src="{spider_chart}" alt="PACT Spider Chart" style="max-width: 600px;">
    </div>
    
    <div class="content">
        {generate_dimension_cards(critique_data)}
    </div>
</body>
</html>
    """
    
    return html

def generate_dimension_cards(critique_data: Dict[str, Any]) -> str:
    """Generate HTML cards for each dimension's detailed analysis."""
    cards_html = ""
    
    for dim_id, critique in critique_data.get('dimension_critiques', {}).items():
        score = critique.get('dimension_score', 0)
        score_class = 'score-high' if score >= 70 else 'score-medium' if score >= 50 else 'score-low'
        
        # Format issues
        issues_html = ""
        for issue in critique.get('issues', []):
            priority_class = f"priority-{issue.get('priority', 'medium').lower()}"
            issues_html += f"""
            <li class="issue-item {priority_class}">
                <strong>{issue.get('title', 'Issue')}</strong><br>
                <small>Location: {issue.get('rubric_id', 'N/A')}</small><br>
                {issue.get('why_it_matters', '')}
                {f"<br><em>Suggestion: {issue.get('rewrite', '')}</em>" if issue.get('rewrite') else ''}
            </li>
            """
        
        cards_html += f"""
        <div class="dimension-card">
            <h3>{critique.get('dimension_name', dim_id)} 
                <span class="score-badge {score_class}">{score:.0f}/100</span>
            </h3>
            <p><strong>Assessment:</strong> {critique.get('overall_assessment', 'N/A')}</p>
            
            <h4>Key Strengths</h4>
            <ul>
                {''.join(f"<li>{s}</li>" for s in critique.get('key_strengths', []))}
            </ul>
            
            <h4>Priority Improvements</h4>
            <ul class="issues-list">
                {issues_html}
            </ul>
        </div>
        """
    
    return cards_html

In [ ]:
%%writefile ../src/pact/enhanced_pdf_generator.py

"""
Enhanced PDF Generation for Comprehensive PACT Analysis Reports

Generates professional PDF reports matching Version 2 quality with spider charts.
"""

import os
from datetime import datetime
from pathlib import Path
from typing import Dict, Any, Optional, List
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO

from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, 
    PageBreak, Image as RLImage, KeepTogether, Flowable
)
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT, TA_JUSTIFY
from reportlab.graphics.shapes import Drawing
from reportlab.graphics.charts.piecharts import Pie
from reportlab.graphics.charts.barcharts import VerticalBarChart

from pact.visualization import create_spider_chart
from pact.enhanced_schemas import ComprehensiveCritique, DetailedDimensionCritique

# Enhanced color scheme
PACT_COLORS = {
    'navy': colors.Color(0.4, 0.49, 0.93),  # #667eea
    'purple': colors.Color(0.46, 0.29, 0.64),  # #764ba2
    'green': colors.Color(0.3, 0.69, 0.31),  # #4caf50
    'orange': colors.Color(1.0, 0.6, 0.0),  # #ff9800
    'red': colors.Color(0.96, 0.26, 0.21),  # #f44336
    'gray': colors.Color(0.5, 0.5, 0.5),
    'light_gray': colors.Color(0.9, 0.9, 0.9)
}

class EnhancedPACTReportGenerator:
    """Professional PDF report generator with visualizations."""
    
    def __init__(self):
        """Initialize the enhanced PDF generator."""
        self._setup_styles()
    
    def _setup_styles(self):
        """Set up comprehensive styles for the report."""
        self.styles = getSampleStyleSheet()
        
        # Title styles
        self.styles.add(ParagraphStyle(
            name='MainTitle',
            parent=self.styles['Title'],
            fontSize=28,
            textColor=PACT_COLORS['navy'],
            spaceAfter=30,
            alignment=TA_CENTER,
            fontName='Helvetica-Bold'
        ))
        
        self.styles.add(ParagraphStyle(
            name='SectionTitle',
            parent=self.styles['Heading1'],
            fontSize=18,
            textColor=PACT_COLORS['navy'],
            spaceBefore=20,
            spaceAfter=12,
            fontName='Helvetica-Bold',
            borderWidth=2,
            borderColor=PACT_COLORS['navy'],
            borderPadding=5
        ))
        
        self.styles.add(ParagraphStyle(
            name='DimensionTitle',
            parent=self.styles['Heading2'],
            fontSize=16,
            textColor=PACT_COLORS['purple'],
            spaceBefore=15,
            spaceAfter=10,
            fontName='Helvetica-Bold'
        ))
        
        # Assessment styles with colors
        self.styles.add(ParagraphStyle(
            name='AssessmentExemplary',
            parent=self.styles['Normal'],
            fontSize=14,
            textColor=PACT_COLORS['green'],
            fontName='Helvetica-Bold'
        ))
        
        self.styles.add(ParagraphStyle(
            name='AssessmentStrong',
            parent=self.styles['Normal'],
            fontSize=14,
            textColor=colors.darkgreen,
            fontName='Helvetica-Bold'
        ))
        
        self.styles.add(ParagraphStyle(
            name='AssessmentCompetent',
            parent=self.styles['Normal'],
            fontSize=14,
            textColor=PACT_COLORS['orange'],
            fontName='Helvetica-Bold'
        ))
        
        self.styles.add(ParagraphStyle(
            name='AssessmentDeveloping',
            parent=self.styles['Normal'],
            fontSize=14,
            textColor=colors.orange,
            fontName='Helvetica-Bold'
        ))
        
        self.styles.add(ParagraphStyle(
            name='AssessmentInadequate',
            parent=self.styles['Normal'],
            fontSize=14,
            textColor=PACT_COLORS['red'],
            fontName='Helvetica-Bold'
        ))
        
        # Enhanced body styles
        self.styles.add(ParagraphStyle(
            name='DetailedAnalysis',
            parent=self.styles['Normal'],
            fontSize=11,
            textColor=colors.black,
            alignment=TA_JUSTIFY,
            spaceAfter=8,
            leading=14
        ))
        
        self.styles.add(ParagraphStyle(
            name='IssueTitle',
            parent=self.styles['Normal'],
            fontSize=12,
            textColor=PACT_COLORS['purple'],
            fontName='Helvetica-Bold',
            spaceAfter=4
        ))
        
        self.styles.add(ParagraphStyle(
            name='Recommendation',
            parent=self.styles['Normal'],
            fontSize=11,
            textColor=colors.darkblue,
            leftIndent=20,
            bulletIndent=10,
            spaceAfter=6
        ))
    
    def generate_comprehensive_report(self, critique_data: Dict[str, Any], 
                                    output_path: str = None) -> str:
        """
        Generate comprehensive PDF report with all enhancements.
        """
        if output_path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = f"PACT_Comprehensive_Report_{timestamp}.pdf"
        
        # Create document
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            rightMargin=60,
            leftMargin=60,
            topMargin=60,
            bottomMargin=60,
            title="PACT Comprehensive Analysis Report",
            author="PACT Academic Analysis System"
        )
        
        # Build content
        story = []
        
        # Page 1: Title and Executive Summary
        story.extend(self._build_title_page(critique_data))
        story.append(PageBreak())
        
        # Page 2: Spider Chart and Overall Scores
        story.extend(self._build_visualization_page(critique_data))
        story.append(PageBreak())
        
        # Page 3: Summary Analysis
        story.extend(self._build_summary_analysis(critique_data))
        story.append(PageBreak())
        
        # Detailed Dimension Analysis (multiple pages)
        for dim_id in ["1.0.0", "2.0.0", "3.0.0", "4.0.0", "5.0.0"]:
            if dim_id in critique_data.get('dimension_critiques', {}):
                story.extend(self._build_dimension_analysis(
                    dim_id, 
                    critique_data['dimension_critiques'][dim_id]
                ))
                story.append(PageBreak())
        
        # Final page: Checklist and Next Steps
        story.extend(self._build_checklist_page(critique_data))
        
        # Build PDF
        doc.build(story, onFirstPage=self._add_header_footer, 
                 onLaterPages=self._add_header_footer)
        
        return output_path
    
    def _build_title_page(self, critique_data: Dict[str, Any]) -> List:
        """Build title page with key information."""
        elements = []
        
        # Title
        elements.append(Paragraph(
            "PACT Comprehensive Analysis Report",
            self.styles['MainTitle']
        ))
        
        elements.append(Spacer(1, 20))
        
        # Document info
        if critique_data.get('paper_title'):
            elements.append(Paragraph(
                f"<b>Document:</b> {critique_data['paper_title']}",
                self.styles['Normal']
            ))
        
        elements.append(Paragraph(
            f"<b>Analysis Date:</b> {datetime.now().strftime('%B %d, %Y')}",
            self.styles['Normal']
        ))
        
        elements.append(Paragraph(
            f"<b>Analysis Type:</b> Comprehensive PACT Assessment",
            self.styles['Normal']
        ))
        
        elements.append(Spacer(1, 30))
        
        # Overall Assessment Box
        overall_score = critique_data.get('overall_score', 0)
        assessment_level = self._get_assessment_level(overall_score)
        assessment_style = f"Assessment{assessment_level}"
        
        assessment_data = [
            ['Overall Assessment', assessment_level],
            ['Overall Score', f"{overall_score:.1f}/100"],
            ['Recommendation', critique_data.get('recommendation', 'See detailed analysis')]
        ]
        
        assessment_table = Table(assessment_data, colWidths=[3*inch, 2.5*inch])
        assessment_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, -1), PACT_COLORS['light_gray']),
            ('TEXTCOLOR', (0, 0), (0, -1), colors.black),
            ('TEXTCOLOR', (1, 0), (1, 0), self._get_assessment_color(assessment_level)),
            ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, -1), 12),
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('GRID', (0, 0), (-1, -1), 1, colors.white),
            ('ROWBACKGROUNDS', (0, 0), (-1, -1), [colors.white, PACT_COLORS['light_gray']])
        ]))
        
        elements.append(assessment_table)
        
        return elements
    
    def _build_visualization_page(self, critique_data: Dict[str, Any]) -> List:
        """Build page with spider chart and dimension scores."""
        elements = []
        
        elements.append(Paragraph("PACT Dimensions Analysis", self.styles['SectionTitle']))
        elements.append(Spacer(1, 20))
        
        # Generate and embed spider chart
        dimension_scores = {}
        for dim_id, critique in critique_data.get('dimension_critiques', {}).items():
            dimension_scores[dim_id] = critique.get('dimension_score', 0)
        
        # Create spider chart image
        spider_chart_base64 = create_spider_chart(dimension_scores)
        
        # Convert base64 to reportlab Image
        # Note: In production, save to temp file and load
        elements.append(Paragraph(
            "Spider Chart Visualization",
            self.styles['Heading3']
        ))
        elements.append(Paragraph(
            "<i>Visual representation of PACT dimension scores showing relative strengths and areas for improvement.</i>",
            self.styles['Normal']
        ))
        
        elements.append(Spacer(1, 20))
        
        # Dimension scores table
        dim_data = [['Dimension', 'Score', 'Assessment', 'Priority']]
        
        dimension_names = {
            "1.0.0": "Research Foundations",
            "2.0.0": "Methodological Rigor", 
            "3.0.0": "Structure & Coherence",
            "4.0.0": "Academic Precision",
            "5.0.0": "Critical Sophistication"
        }
        
        for dim_id, name in dimension_names.items():
            if dim_id in dimension_scores:
                score = dimension_scores[dim_id]
                assessment = self._get_assessment_level(score)
                priority = "High" if score < 50 else "Medium" if score < 70 else "Low"
                dim_data.append([name, f"{score:.0f}", assessment, priority])
        
        dim_table = Table(dim_data, colWidths=[2.5*inch, 1*inch, 1.5*inch, 1*inch])
        dim_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), PACT_COLORS['navy']),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, -1), 10),
            ('ALIGN', (1, 0), (-1, -1), 'CENTER'),
            ('GRID', (0, 0), (-1, -1), 1, colors.gray),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, PACT_COLORS['light_gray']])
        ]))
        
        elements.append(dim_table)
        
        return elements
    
    def _build_summary_analysis(self, critique_data: Dict[str, Any]) -> List:
        """Build summary analysis section."""
        elements = []
        
        elements.append(Paragraph("Summary Analysis", self.styles['SectionTitle']))
        elements.append(Spacer(1, 12))
        
        # Executive Summary
        elements.append(Paragraph("<b>Executive Summary:</b>", self.styles['Heading3']))
        summary_text = critique_data.get('executive_summary', 
            'This work demonstrates competent academic writing with opportunities for targeted improvement.')
        elements.append(Paragraph(summary_text, self.styles['DetailedAnalysis']))
        elements.append(Spacer(1, 12))
        
        # Key Strengths
        elements.append(Paragraph("<b>Key Strengths:</b>", self.styles['Heading3']))
        strengths = critique_data.get('key_strengths', [])
        for strength in strengths[:5]:  # Top 5 strengths
            elements.append(Paragraph(f"• {strength}", self.styles['Recommendation']))
        elements.append(Spacer(1, 12))
        
        # Priority Improvements
        elements.append(Paragraph("<b>Priority Areas for Improvement:</b>", self.styles['Heading3']))
        improvements = critique_data.get('priority_improvements', [])
        for improvement in improvements[:5]:  # Top 5 improvements
            elements.append(Paragraph(f"• {improvement}", self.styles['Recommendation']))
        
        return elements
    
    def _build_dimension_analysis(self, dim_id: str, critique: Dict[str, Any]) -> List:
        """Build detailed analysis for a single dimension."""
        elements = []
        
        # Dimension header
        dim_name = critique.get('dimension_name', f'Dimension {dim_id}')
        score = critique.get('dimension_score', 0)
        assessment = critique.get('overall_assessment', 'Competent')
        
        elements.append(Paragraph(
            f"{dim_name} (Score: {score:.0f}/100)",
            self.styles['DimensionTitle']
        ))
        
        # Assessment level
        assessment_style = f"Assessment{self._get_assessment_level(score)}"
        elements.append(Paragraph(
            f"Assessment: {assessment}",
            self.styles[assessment_style]
        ))
        elements.append(Spacer(1, 12))
        
        # Detailed analysis
        if 'comprehensive_assessment' in critique:
            elements.append(Paragraph(
                critique['comprehensive_assessment'],
                self.styles['DetailedAnalysis']
            ))
            elements.append(Spacer(1, 12))
        
        # Issues with structured format
        if critique.get('issues'):
            elements.append(Paragraph("<b>Specific Issues:</b>", self.styles['Heading3']))
            
            for issue in critique['issues'][:6]:  # Top 6 issues
                # Issue title with priority
                priority_color = self._get_priority_color(issue.get('priority', 'Standard'))
                elements.append(Paragraph(
                    f"<font color='{priority_color}'>■</font> <b>{issue.get('title', 'Issue')}</b>",
                    self.styles['IssueTitle']
                ))
                
                # Why it matters
                elements.append(Paragraph(
                    f"<i>Why it matters:</i> {issue.get('why_it_matters', '')}",
                    self.styles['Normal']
                ))
                
                # Evidence if available
                if issue.get('evidence'):
                    elements.append(Paragraph(
                        f"<i>Evidence:</i> \"{issue['evidence'][0] if issue['evidence'] else ''}\"",
                        ParagraphStyle(
                            name='Evidence',
                            parent=self.styles['Normal'],
                            fontSize=10,
                            textColor=colors.gray,
                            leftIndent=20
                        )
                    ))
                
                # Suggestion if available
                if issue.get('rewrite'):
                    elements.append(Paragraph(
                        f"<i>Suggestion:</i> {issue['rewrite']}",
                        self.styles['Recommendation']
                    ))
                
                elements.append(Spacer(1, 8))
        
        return elements
    
    def _build_checklist_page(self, critique_data: Dict[str, Any]) -> List:
        """Build final checklist page."""
        elements = []
        
        elements.append(Paragraph("PACT Improvement Checklist", self.styles['SectionTitle']))
        elements.append(Spacer(1, 12))
        
        elements.append(Paragraph(
            "Use this checklist to track your revision progress:",
            self.styles['Normal']
        ))
        elements.append(Spacer(1, 12))
        
        # Create checklist items from all dimension issues
        checklist_data = []
        
        for dim_id, critique in critique_data.get('dimension_critiques', {}).items():
            dim_name = critique.get('dimension_name', dim_id)
            
            for issue in critique.get('issues', [])[:3]:  # Top 3 per dimension
                checklist_data.append([
                    '☐',
                    dim_name,
                    issue.get('title', 'Issue'),
                    issue.get('priority', 'Standard')
                ])
        
        if checklist_data:
            checklist_table = Table(
                [['', 'Dimension', 'Issue', 'Priority']] + checklist_data,
                colWidths=[0.3*inch, 1.8*inch, 3*inch, 0.8*inch]
            )
            
            checklist_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), PACT_COLORS['navy']),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, -1), 9),
                ('ALIGN', (0, 0), (0, -1), 'CENTER'),
                ('ALIGN', (-1, 0), (-1, -1), 'CENTER'),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.gray),
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, PACT_COLORS['light_gray']])
            ]))
            
            elements.append(checklist_table)
        
        elements.append(Spacer(1, 20))
        
        # Next steps
        elements.append(Paragraph("<b>Recommended Next Steps:</b>", self.styles['Heading3']))
        next_steps = critique_data.get('actionable_next_steps', [
            "Address high-priority issues first",
            "Review and incorporate suggested rewrites",
            "Seek feedback after implementing changes",
            "Re-evaluate using PACT criteria"
        ])
        
        for i, step in enumerate(next_steps[:5], 1):
            elements.append(Paragraph(f"{i}. {step}", self.styles['Recommendation']))
        
        return elements
    
    def _get_assessment_level(self, score: float) -> str:
        """Convert numeric score to assessment level."""
        if score >= 85:
            return "Exemplary"
        elif score >= 70:
            return "Strong"
        elif score >= 55:
            return "Competent"
        elif score >= 40:
            return "Developing"
        else:
            return "Inadequate"
    
    def _get_assessment_color(self, level: str) -> colors.Color:
        """Get color for assessment level."""
        color_map = {
            "Exemplary": PACT_COLORS['green'],
            "Strong": colors.darkgreen,
            "Competent": PACT_COLORS['orange'],
            "Developing": colors.orange,
            "Inadequate": PACT_COLORS['red']
        }
        return color_map.get(level, colors.black)
    
    def _get_priority_color(self, priority: str) -> str:
        """Get hex color for priority level."""
        color_map = {
            "Critical": "#f44336",
            "High": "#ff9800",
            "Medium": "#ffc107",
            "Standard": "#4caf50",
            "Low": "#4caf50"
        }
        return color_map.get(priority, "#666666")
    
    def _add_header_footer(self, canvas, doc):
        """Add header and footer to each page."""
        canvas.saveState()
        
        # Header
        canvas.setFont('Helvetica', 9)
        canvas.setFillColor(colors.gray)
        canvas.drawString(inch, doc.height + doc.topMargin - 0.5*inch, 
                         "PACT Academic Analysis Report")
        canvas.drawRightString(doc.width + doc.rightMargin, 
                              doc.height + doc.topMargin - 0.5*inch,
                              datetime.now().strftime("%B %Y"))
        
        # Footer
        page_num = canvas.getPageNumber()
        canvas.drawCentredString(doc.width/2 + doc.leftMargin, 
                                0.5*inch,
                                f"Page {page_num}")
        
        canvas.restoreState()

## Usage Examples

In [ ]:
# Example: Process a paper from file
# Uncomment and modify path as needed
"""
from pact.pact_file_processor import critique_paper_file

# Critique a paper from a file
critique = await critique_paper_file(
    file_path="path/to/student_paper.pdf",
    output_dir="critique_reports/"
)

print("Critique generated successfully!")
"""

In [ ]:
# Create a simple CLI interface
%%writefile ../src/pact/pact_cli.py

"""
Command-line interface for PACT Critique System
"""

import asyncio
import argparse
from pathlib import Path

async def main():
    parser = argparse.ArgumentParser(description='PACT Academic Paper Critique System')
    parser.add_argument('input_file', help='Path to the paper file to critique')
    parser.add_argument('-o', '--output', help='Output directory for critique report')
    parser.add_argument('-f', '--format', choices=['md', 'txt', 'html'], 
                       default='md', help='Output format for critique')
    
    args = parser.parse_args()
    
    print(f"Processing paper: {args.input_file}")
    print("This may take a few minutes...")
    
    from pact.pact_file_processor import critique_paper_file
    
    try:
        critique = await critique_paper_file(
            file_path=args.input_file,
            output_dir=args.output
        )
        print("\n" + "="*50)
        print("Critique Complete!")
        print("="*50)
        
        if not args.output:
            print("\n" + critique)
    except Exception as e:
        print(f"Error: {e}")
        return 1
    
    return 0

if __name__ == "__main__":
    asyncio.run(main())

## Summary

This notebook has created a comprehensive PACT-based multi-agent critique system that:

1. **Loads and parses the PACT taxonomy** from the provided PACT_JSON.docx file
2. **Implements 5 specialized agents**, each focused on one PACT dimension
3. **Uses a supervisor agent** to coordinate the critique and synthesize feedback
4. **Provides structured, actionable feedback** with scores and recommendations
5. **Supports multiple file formats** for paper input (PDF, DOCX, TXT, MD)
6. **Generates formatted reports** in multiple output formats

The system can be:
- Used interactively in this notebook
- Deployed as a web service using LangGraph
- Run from the command line using the CLI interface
- Integrated into existing academic workflows

The modular design allows for easy customization and extension, such as:
- Adding more specialized agents for specific paper types
- Customizing rubrics for different academic levels
- Integrating with learning management systems
- Adding batch processing capabilities